In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [144]:
# Brainstorming

# Stats
stat_names = [
    "Strength",
    "Agility",
    "Intelligence",
    "Wisdom",
    "Stamina",
    "Crit_Chance",
    "Crit_Bonus",
    "Potency",
    "Primary_Stat",
    "Ability_Mod",
    "Cast_%",
    "Recovery_%",
    "Reuse_%",
    "Spell_Double_Attack",
    "Ability_Double_Attack",
    "DPS_Mod",
    "Haste_Mod",
    "MA_Mod",
    "Flurry_Mod",
]


# Feature_Engineering
engineered_columns = [
    "ticks_per_second",
    "Init_Crit_Min",
    "Init_Crit_Max",
    "Init_Crit_Avg",
    "Dot_Crit_Min",
    "Dot_Crit_Max",
    "Dot_Crit_Average"
    "Damage_Total_Min",
    "Damage_Total_Max",
    "Damage_Total_Average",
    "Cast_Efficiency",
    "Average_DPS",
    "Spam_DPS",
    "Spam_Uptime"
]


# Abilities
ability_column_names = [
    "Ability_Name",
    "Ability_Type", # Attack, Heal, Buff, Rez, 
    "Max_Targets", # 1 (default), 0 for unlimited (rare cases), 8 for most blue
    "Target_Type", # Single (red) (default), Enc_AOE (green), PB_AOE (blue), T_AOE (Blue), Self (Yellow), Group (Purple), Raid (Purple)
    "Cast_Time", # 
    "Reuse_Time",
    "Recovery_Time", # 0.5 (default)
    "Dmg_Init_Min",
    "Dmg_Init_Max",
    "Dmg_Tick_Min",
    "Dmg_Tick_Max",
    "Tick_Rate",    
]

ability_kwargs = [
    "Add_CB_Bonus",
    "Position_Requirements",
    "Stealth_Requirements",
    "Threat_Min",
    "Threat_Max",
    "Threat_Position",
    "Range",
    "Mit_Debuff",
]




# Brainstorming

In [145]:
# Import_abilities
df = pd.read_csv("sample_abilities.txt", sep="\t")
df.head()

,AE? (1=Y),Spell,Cast,Reuse,Min,Max,Dot min,Dot max,additional multiplier,Duration,s/tick,tick/s
0,NaN,Exploit Weakness,0.25,6.0,4142113,6363636,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Dagger Storm,0.25,15.0,702820,774482,702820.0,774482.0,NaN,11.0,1.0,1.0
2,NaN,Nightblade,0.00,5.0,872528,1215306,194533.0,361275.0,NaN,2.0,1.0,1.0
3,NaN,Ambush,0.25,5.0,771232,1000310,495241.0,724320.0,15.00%,1.0,1.0,1.0
4,NaN,Ambush+Masked,0.75,5.0,1382585,1766787,830603.0,1214807.0,NaN,NaN,NaN,NaN


# Cleaning

In [146]:
rename_ability_cols = [
    "AoE", # 
    "Ability_Name", # Spell
    "Cast_Time", # Cast
    "Reuse_Time", # Reuse
    "Dmg_Init_Min", # Min
    "Dmg_Init_Max", # Max
    "Dmg_Tick_Min", # Dot min
    "Dmg_Tick_Max", # Dot max
    "Addl_Crit_Bonus", # additional multiplier
    "Duration", # duration
    "Tick_Rate", # s/tick
    "Ticks_per_sec",
]
# Rename columns
df.columns=rename_ability_cols

# fill in missing columns and values
df.fillna(0,inplace=True)
df["Recovery_Time"] = 0.25
df['AoE']=df['AoE'].astype('bool')
clean_percents = lambda x : float(str(x).strip("%"))//100

df[["Dmg_Tick_Min", "Dmg_Tick_Max"]]=df[["Dmg_Tick_Min", "Dmg_Tick_Max"]].astype("Int32")

df["Addl_Crit_Bonus"] = df["Addl_Crit_Bonus"].apply(clean_percents)
display(df)
# '''
# need more info for these ones

#     "Ability_Type", # Attack, Heal, Buff, Rez, 
#     "Max_Targets", 
#     "Target_Type", 

# '''

,AoE,Ability_Name,Cast_Time,Reuse_Time,Dmg_Init_Min,Dmg_Init_Max,Dmg_Tick_Min,Dmg_Tick_Max,Addl_Crit_Bonus,Duration,Tick_Rate,Ticks_per_sec,Recovery_Time
0,False,Exploit Weakness,0.250,6.0,4142113,6363636,0,0,0.0,0.0,0.0,0.000000,0.25
1,False,Dagger Storm,0.250,15.0,702820,774482,702820,774482,0.0,11.0,1.0,1.000000,0.25
2,False,Nightblade,0.000,5.0,872528,1215306,194533,361275,0.0,2.0,1.0,1.000000,0.25
3,False,Ambush,0.250,5.0,771232,1000310,495241,724320,0.0,1.0,1.0,1.000000,0.25
4,False,Ambush+Masked,0.750,5.0,1382585,1766787,830603,1214807,0.0,0.0,0.0,0.000000,0.25
5,True,AcidMist,0.000,5.0,389066,722550,389066,722550,0.0,1.0,1.0,1.000000,0.25
6,False,Bleedout,0.250,15.0,805002,1087056,43644,72740,0.0,24.0,1.0,1.000000,0.25
7,False,BloodflurryMax,0.250,10.0,567498,735002,291508,459012,0.0,4.0,1.0,1.000000,0.25
8,False,DeathMark,0.125,15.0,322722,472000,322722,472000,0.0,7.0,1.0,1.000000,0.25
9,False,Masked Strike,0.250,5.0,611353,766477,335362,490487,0.0,1.0,1.0,1.000000,0.25


In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   AoE              41 non-null     bool   
 1   Ability_Name     41 non-null     object 
 2   Cast_Time        41 non-null     float64
 3   Reuse_Time       41 non-null     float64
 4   Dmg_Init_Min     41 non-null     int64  
 5   Dmg_Init_Max     41 non-null     int64  
 6   Dmg_Tick_Min     41 non-null     Int32  
 7   Dmg_Tick_Max     41 non-null     Int32  
 8   Addl_Crit_Bonus  41 non-null     float64
 9   Duration         41 non-null     float64
 10  Tick_Rate        41 non-null     float64
 11  Ticks_per_sec    41 non-null     float64
 12  Recovery_Time    41 non-null     float64
dtypes: Int32(2), bool(1), float64(7), int64(2), object(1)
memory usage: 3.8+ KB


In [148]:
def feature_engineering(ability_df, character_stats=None):
    '''
    Just here for 
    ability_columns = [
        "Total_Ticks",
        "Init_Crit_Min",
        "Init_Crit_Max",
        "Init_Crit_Avg",
        "Dot_Crit_Min",
        "Dot_Crit_Max",
        "Dot_Crit_Average"
        "Damage_Total_Min",
        "Damage_Total_Max",
        "Damage_Total_Average",
        "Cast_Efficiency",
        "Average_DPS",
        "Spam_DPS",
        "Spam_Uptime",
    ]
    
    column_names = [
        "Ability_Name", # Spell
        "Cast_Time", # Cast
        "Reuse_Time", # Reuse
        "Dmg_Init_Min", # Min
        "Dmg_Init_Max", # Max
        "Dmg_Tick_Min", # Dot min
        "Dmg_Tick_Max", # Dot max
        "Addl_Crit_Bonus", # additional multiplier
        "Duration", # duration
        "Tick_Rate", # s/tick
        "Ticks_per_sec",
        "Recovery_Time", # Recovery
    ]
    stat_names = [
        "Strength",
        "Agility",
        "Intelligence",
        "Wisdom",
        "Stamina",
        "Crit_Chance",
        "Crit_Bonus",
        "Potency",
        "Primary_Stat",
        "Ability_Mod",
        "Cast_%",
        "Recovery_%",
        "Reuse_%",
        "Spell_Double_Attack",
        "Ability_Double_Attack",
        "DPS_Mod",
        "Haste_Mod",
        "MA_Mod",
        "Flurry_Mod",
    ]
    '''
    character_stats = {
        "Crit_Bonus": float(3.122),
        "Recovery_%": 1.00,        
    }
    df = pd.DataFrame(ability_df)
    df['Total_Ticks'] = np.where(df['Tick_Rate'] > 0, df['Duration'] / df['Tick_Rate'], 0).astype("int")
    df["Init_Crit_Min"] =  df.apply(lambda x: max(x['Dmg_Init_Min']*(x['Addl_Crit_Bonus']+character_stats['Crit_Bonus']), x['Dmg_Init_Max']+1), axis=1)
    
    df["Init_Crit_Max"] = df['Dmg_Init_Max']*(character_stats['Crit_Bonus']+df['Addl_Crit_Bonus'])
    # df["Init_Crit_Avg"] = 
    df["Dot_Crit_Min"] = np.where(
        df["Dmg_Tick_Min"] * (character_stats['Crit_Bonus'] + df['Addl_Crit_Bonus']) < (df["Dmg_Tick_Max"] + 1),
        np.where(
            df["Dmg_Tick_Max"] + 1 <= 1,
            0,
            df["Dmg_Tick_Max"] + 1
        ),
        df["Dmg_Tick_Min"] * (df['Addl_Crit_Bonus'] + character_stats['Crit_Bonus'])
        ).astype("int")

    df["Dot_Crit_Max"] = (df["Dmg_Tick_Max"]*(character_stats['Crit_Bonus']+df['Addl_Crit_Bonus'])).astype("int")
    # df["Dot_Crit_Average"] = 
    # df["Damage_Total_Min"] = 
    df["Dot_Crit_Max_Total"] = (df['Init_Crit_Max']+df['Dot_Crit_Max']*df['Ticks_per_sec']*df['Duration']).astype("int")
    # df["Damage_Total_Average"] = (if($A8=1,$B$4,1)*((M8*df['Dmg_Init_Min']+df['Dmg_Init_Max']*df['Dot_Crit_Max'])/(df['Dmg_Init_Min']+df['Dmg_Init_Max'])+if(df["Dmg_Tick_Min"]>0,(O8*df["Dmg_Tick_Min"]+df["Dmg_Tick_Max"]*df['Dot_Crit_Max'])/(df["Dmg_Tick_Min"]+df["Dmg_Tick_Max"])*df['Duration']*df['Ticks_per_sec'],0)))
    # df["Cast_Efficiency"] =R8/(C8+0.5*(1/(1+$B$5)))*if(V8=1,1,1+$B$3)
    # df["Average_DPS"] = $R8/(($C8+$D8))*if(V8=1,1,1+$B$3)
    # df["Spam_DPS"] = 
    # df["Spam_Uptime"] = 
    df.head()
    return df
feature_engineering(ability_df=df)

,AoE,Ability_Name,Cast_Time,Reuse_Time,Dmg_Init_Min,Dmg_Init_Max,Dmg_Tick_Min,Dmg_Tick_Max,Addl_Crit_Bonus,Duration,Tick_Rate,Ticks_per_sec,Recovery_Time,Total_Ticks,Init_Crit_Min,Init_Crit_Max,Dot_Crit_Min,Dot_Crit_Max,Dot_Crit_Max_Total
0,False,Exploit Weakness,0.250,6.0,4142113,6363636,0,0,0.0,0.0,0.0,0.000000,0.25,0,1.293168e+07,1.986727e+07,0,0,19867271
1,False,Dagger Storm,0.250,15.0,702820,774482,702820,774482,0.0,11.0,1.0,1.000000,0.25,11,2.194204e+06,2.417933e+06,2194204,2417932,29015184
2,False,Nightblade,0.000,5.0,872528,1215306,194533,361275,0.0,2.0,1.0,1.000000,0.25,2,2.724032e+06,3.794185e+06,607332,1127900,6049985
3,False,Ambush,0.250,5.0,771232,1000310,495241,724320,0.0,1.0,1.0,1.000000,0.25,1,2.407786e+06,3.122968e+06,1546142,2261327,5384294
4,False,Ambush+Masked,0.750,5.0,1382585,1766787,830603,1214807,0.0,0.0,0.0,0.000000,0.25,0,4.316430e+06,5.515909e+06,2593142,3792627,5515909
5,True,AcidMist,0.000,5.0,389066,722550,389066,722550,0.0,1.0,1.0,1.000000,0.25,1,1.214664e+06,2.255801e+06,1214664,2255801,4511602
6,False,Bleedout,0.250,15.0,805002,1087056,43644,72740,0.0,24.0,1.0,1.000000,0.25,24,2.513216e+06,3.393789e+06,136256,227094,8844044
7,False,BloodflurryMax,0.250,10.0,567498,735002,291508,459012,0.0,4.0,1.0,1.000000,0.25,4,1.771729e+06,2.294676e+06,910087,1433035,8026816
8,False,DeathMark,0.125,15.0,322722,472000,322722,472000,0.0,7.0,1.0,1.000000,0.25,7,1.007538e+06,1.473584e+06,1007538,1473584,11788672
9,False,Masked Strike,0.250,5.0,611353,766477,335362,490487,0.0,1.0,1.0,1.000000,0.25,1,1.908644e+06,2.392941e+06,1047000,1531300,3924241


## Feature Engineering